In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
import sys

# Lambda_idx = int(sys.argv[1])
# n_external = int(sys.argv[2])
Lambda_idx = 18
n_external = 3


Lambda = Lambdas[Lambda_idx]

In [2]:
# CCAT-prime
zmin = 3.5
zmax = 8.1

Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

In [3]:
from scipy.interpolate import interp1d
from scipy.integrate import quad, quad_vec, trapezoid

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
f_inner_integral = interp1d(x = chibs, y = inner_dkparp_integral, axis = 1)

In [4]:
external_chis = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), n_external)
print('external chi spacing', np.mean(np.diff(external_chis)))

external chi spacing 1104.768931326645


In [5]:
ILoILo = np.zeros((len(ells), (n_external), (n_external)), dtype=np.float64)

In [6]:
from tqdm import trange

In [7]:
# for chi_idx in trange(n_external):
#     chi = external_chis[chi_idx]
#     chip = external_chis

#     f_KLIMLo1 = get_f_KILo(external_chi = chi, Lambda=Lambda)
#     f_KLIMLo1_windowed = apply_window(f_K = f_KLIMLo1,
#                                      chimin = chimin,
#                                      chimax = chimax)

#     f_KLIMLo2 = get_f_KILo(external_chi = chip.reshape(n_external, 1), Lambda=Lambda)
#     f_KLIMLo2_windowed = apply_window(f_K = f_KLIMLo2,
#                                      chimin = chimin,
#                                      chimax = chimax)


#     def integrand(_chib):
#         plus = _chib*(1+deltas)
#         mins = _chib*(1-deltas)

#         _interm  = f_KLIMLo1_windowed(plus) * f_KLIMLo2_windowed(mins)
#         _interm += f_KLIMLo1_windowed(mins) * f_KLIMLo2_windowed(plus) 

#         _factor = (2 / _chib)
#         _factor = _factor * deltas
#         _factor = np.einsum('d, ld->ld', _factor, f_inner_integral(_chib))

#         _interm  = np.einsum('ed,ld->led', _interm, _factor)

#         return trapezoid(x = np.log(deltas), y = _interm, axis=-1)

#     ILoILo[:,chi_idx,:], _ = quad_vec(integrand, 10, chimax_sample,
#                                       epsrel = 1e-3, epsabs =0.0)

In [8]:
f_KLIMLo1 = get_f_KILo(external_chi = external_chis.reshape(n_external,1,1), Lambda=Lambda)
f_KLIMLo1_windowed = apply_window(f_K = f_KLIMLo1,
                                 chimin = chimin,
                                 chimax = chimax)

f_KLIMLo2 = get_f_KILo(external_chi = external_chis.reshape(1,n_external, 1), Lambda=Lambda)
f_KLIMLo2_windowed = apply_window(f_K = f_KLIMLo2,
                                 chimin = chimin,
                                 chimax = chimax)


def integrand(_chib):
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)

    _interm  = f_KLIMLo1_windowed(plus) * f_KLIMLo2_windowed(mins)
    _interm += f_KLIMLo1_windowed(mins) * f_KLIMLo2_windowed(plus) 

    _factor = (2 / _chib)
    _factor = _factor * deltas
    _factor = np.einsum('d, ld->ld', _factor, f_inner_integral(_chib))

    _interm  = np.einsum('xyd,ld->lxyd', _interm, _factor)

    return trapezoid(x = np.log(deltas), y = _interm, axis=-1)
res, _ = quad_vec(integrand, 10, chimax_sample, epsrel = 1e-3, epsabs =0.0)

In [11]:
print(res)

[[[ 6.59523781e-07 -5.31186709e-08  3.06414933e-08]
  [-5.31186709e-08  8.32985655e-08 -4.39534693e-09]
  [ 3.06414933e-08 -4.39534693e-09  4.87180620e-09]]

 [[ 6.61729436e-07 -5.29839749e-08  3.07211017e-08]
  [-5.29839749e-08  8.36338859e-08 -4.38842441e-09]
  [ 3.07211017e-08 -4.38842441e-09  4.87635571e-09]]

 [[ 6.64115027e-07 -5.28481404e-08  3.08072636e-08]
  [-5.28481404e-08  8.39979500e-08 -4.38150930e-09]
  [ 3.08072636e-08 -4.38150930e-09  4.88130278e-09]]

 [[ 6.66690839e-07 -5.27118231e-08  3.09004071e-08]
  [-5.27118231e-08  8.43926423e-08 -4.37464853e-09]
  [ 3.09004071e-08 -4.37464853e-09  4.88667662e-09]]

 [[ 6.69466928e-07 -5.25756901e-08  3.10009640e-08]
  [-5.25756901e-08  8.48198662e-08 -4.36789156e-09]
  [ 3.10009640e-08 -4.36789156e-09  4.89250715e-09]]

 [[ 6.72452905e-07 -5.24404006e-08  3.11093631e-08]
  [-5.24404006e-08  8.52815239e-08 -4.36128916e-09]
  [ 3.11093631e-08 -4.36128916e-09  4.89882460e-09]]

 [[ 6.75657654e-07 -5.23065884e-08  3.12260203e-08]


In [12]:
res.shape

(100, 3, 3)